# ETL Weather DFs

In [79]:
import pandas as pd
import os
import numpy as np
import re

### Aggregate station data by distance

In [31]:
pd.options.display.max_rows = 10
cities = ["ATL_stations","CH_stations","LA_stations","NYC_stations","SD_stations","SF_stations"]
#Aggregate Station Location tuples 
stations_data = pd.DataFrame()
for city in cities:
    path = 'station_locations/%s.txt' % city
    if os.path.exists(path):
        frame = pd.read_csv(path,delim_whitespace=True,header=None,error_bad_lines=False)
        frame['city'] = city
        stations_data = stations_data.append(frame,ignore_index=True)
stations_data = stations_data.rename(columns={0:"distance",1:"station_name"})

Skipping line 33: expected 9 fields, saw 10
Skipping line 44: expected 9 fields, saw 10
Skipping line 45: expected 9 fields, saw 10
Skipping line 53: expected 9 fields, saw 10
Skipping line 56: expected 9 fields, saw 10
Skipping line 60: expected 9 fields, saw 10
Skipping line 83: expected 9 fields, saw 10
Skipping line 84: expected 9 fields, saw 10
Skipping line 86: expected 9 fields, saw 10
Skipping line 90: expected 9 fields, saw 10
Skipping line 111: expected 9 fields, saw 10
Skipping line 112: expected 9 fields, saw 10
Skipping line 114: expected 9 fields, saw 10
Skipping line 115: expected 9 fields, saw 10
Skipping line 118: expected 9 fields, saw 10
Skipping line 119: expected 9 fields, saw 10
Skipping line 125: expected 9 fields, saw 10
Skipping line 127: expected 9 fields, saw 10
Skipping line 138: expected 9 fields, saw 10
Skipping line 140: expected 9 fields, saw 10
Skipping line 144: expected 9 fields, saw 10
Skipping line 146: expected 9 fields, saw 10
Skipping line 153: e

In [32]:
ATLshortlist = stations_data[(stations_data.city == "ATL_stations") & (stations_data.distance <= 100)][['station_name']]
CHshortlist = stations_data[(stations_data.city == "CH_stations") & (stations_data.distance <= 100)][['station_name']]
LAshortlist = stations_data[(stations_data.city == "LA_stations") & (stations_data.distance <= 100)][['station_name']]
NYCshortlist = stations_data[(stations_data.city == "NYC_stations") & (stations_data.distance <= 100)][['station_name']]
SDshortlist = stations_data[(stations_data.city == "SD_stations") & (stations_data.distance <= 100)][['station_name']]
SFshortlist = stations_data[(stations_data.city == "SF_stations") & (stations_data.distance <= 100)][['station_name']]

### Transform and Load Data 
###### Merge datasets from each station with PARAM_STATION-NAME as default column header
##### Process results in 10 stations per city
##### UPDATE: Frame.query removes scrappy data (missing data will still exist for some!) 

In [63]:
######ATL Weather######
ATL_stations = [];
ATL_weather = pd.DataFrame()
for station in ATLshortlist['station_name']:
    path = 'ATL/%s.csv' % station
    if os.path.exists(path):
        frame = pd.read_csv(path,delim_whitespace=True)
        frame = frame.query('TMIN != TMAX')
        frame.columns = ['DATE', 'TMAX_'+station,'TMIN_'+station,'SNOW_'+station,'SNWD_'+station,'PRCP_'+station]
        if frame.shape[0] >= 12650:
            ATL_stations.append(station);
            if ATL_weather.empty:
                ATL_weather = ATL_weather.append(frame,ignore_index=True)
            else:
                ATL_weather = ATL_weather.merge(frame, on='DATE', how='inner', suffixes=('',''))
ATL_weather = ATL_weather.groupby('DATE').mean()
ATL_weather.reset_index(level=0, inplace=True)

In [61]:
######CH Weather######
CH_stations = [];
CH_weather = pd.DataFrame()
for station in CHshortlist['station_name']:
    path = 'CH/%s.csv' % station
    if os.path.exists(path):
        frame = pd.read_csv(path,delim_whitespace=True)
        frame = frame.query('TMIN != TMAX')
        frame.columns = ['DATE', 'TMAX_'+station,'TMIN_'+station,'SNOW_'+station,'SNWD_'+station,'PRCP_'+station]
        if frame.shape[0] >= 12764:
            CH_stations.append(station)
            if CH_weather.empty:
                CH_weather = CH_weather.append(frame,ignore_index=True)
            else:
                CH_weather = CH_weather.merge(frame, on='DATE', how='inner', suffixes=('',''))
CH_weather = CH_weather.groupby('DATE').mean()
CH_weather.reset_index(level=0, inplace=True)

In [64]:
######NYC Weather######
NYC_stations = [];
NYC_weather = pd.DataFrame()
for station in NYCshortlist['station_name']:
    path = 'NYC/%s.csv' % station
    if os.path.exists(path):
        frame = pd.read_csv(path,delim_whitespace=True)
        frame = frame.query('TMIN != TMAX')
        frame.columns = ['DATE', 'TMAX_'+station,'TMIN_'+station,'SNOW_'+station,'SNWD_'+station,'PRCP_'+station]
        if frame.shape[0] >= 12700:
            NYC_stations.append(station)
            if NYC_weather.empty:
                NYC_weather = NYC_weather.append(frame,ignore_index=True)
            else:
                NYC_weather = NYC_weather.merge(frame, on='DATE', how='inner', suffixes=('',''))
NYC_weather = NYC_weather.groupby('DATE').mean()
NYC_weather.reset_index(level=0, inplace=True)

In [65]:
######LA Weather######
LA_stations = [];
LA_weather = pd.DataFrame()
for station in LAshortlist['station_name']:
    path = 'LA/%s.csv' % station
    if os.path.exists(path):
        frame = pd.read_csv(path,delim_whitespace=True)
        frame = frame.query('TMIN != TMAX')
        frame.columns = ['DATE', 'TMAX_'+station,'TMIN_'+station,'SNOW_'+station,'SNWD_'+station,'PRCP_'+station]
        if frame.shape[0] >= 12600: 
            LA_stations.append(station)
            if LA_weather.empty:
                LA_weather = LA_weather.append(frame,ignore_index=True)
            else:
                LA_weather = LA_weather.merge(frame, on='DATE', how='inner', suffixes=('',''))
LA_weather = LA_weather.groupby('DATE').mean()
LA_weather.reset_index(level=0, inplace=True)

In [66]:
######SF Weather######
SF_stations = [];
SF_weather = pd.DataFrame()
for station in SFshortlist['station_name']:
    path = 'SF/%s.csv' % station
    if os.path.exists(path):
        frame = pd.read_csv(path,delim_whitespace=True)
        frame = frame.query('TMIN != TMAX')
        frame.columns = ['DATE', 'TMAX_'+station,'TMIN_'+station,'SNOW_'+station,'SNWD_'+station,'PRCP_'+station]
        if frame.shape[0] >= 12784: 
            SF_stations.append(station)
            if SF_weather.empty:
                SF_weather = SF_weather.append(frame,ignore_index=True)
            else:
                SF_weather = SF_weather.merge(frame, on='DATE', how='inner', suffixes=('',''))
SF_weather = SF_weather.groupby('DATE').mean()
SF_weather.reset_index(level=0, inplace=True)

In [67]:
######SD Weather######
SD_stations = [];
SD_weather = pd.DataFrame()
for station in SDshortlist['station_name']:
    path = 'SD/%s.csv' % station
    if os.path.exists(path):
        frame = pd.read_csv(path,delim_whitespace=True)
        frame = frame.query('TMIN != TMAX')
        frame.columns = ['DATE', 'TMAX_'+station,'TMIN_'+station,'SNOW_'+station,'SNWD_'+station,'PRCP_'+station]
        if frame.shape[0] >= 12600: 
            SD_stations.append(station)
            if SD_weather.empty:
                SD_weather = SD_weather.append(frame,ignore_index=True)
            else:
                SD_weather = SD_weather.merge(frame, on='DATE', how='inner', suffixes=('',''))
SD_weather = SD_weather.groupby('DATE').mean()
SD_weather.reset_index(level=0, inplace=True)

### Missing Data Logger
##### True indicates a missing data row 

In [39]:
print("New York City stations missing data")
for station in NYC_stations:
    print("Station %s" % station)
    print(pd.isnull(NYC_weather)['TMIN_'+station].value_counts())
    print()

print("Atlanta stations missing data")
for station in ATL_stations:
    print("Station %s" % station)
    print(pd.isnull(ATL_weather)['TMIN_'+station].value_counts())
    print()
    
print("San Francisco stations missing data")
for station in SF_stations:
    print("Station %s" % station)
    print(pd.isnull(SF_weather)['TMIN_'+station].value_counts())
    print()

print("San Diego stations missing data")
for station in SD_stations:
    print("Station %s" % station)
    print(pd.isnull(SD_weather)['TMIN_'+station].value_counts())
    print()

print("Los Angeles stations missing data")
for station in LA_stations:
    print("Station %s" % station)
    print(pd.isnull(LA_weather)['TMIN_'+station].value_counts())
    print()

print("Chicago stations missing data")
for station in CH_stations:
    print("Station %s" % station)
    print(pd.isnull(CH_weather)['TMIN_'+station].value_counts())
    print()    

New York City stations missing data
Station USC00300889
False    12631
dtype: int64
()
Station USC00305426
False    12631
dtype: int64
()
Station USW00094745
False    12631
dtype: int64
()
Station USC00283951
False    12631
dtype: int64
()
Station USW00014734
False    12631
dtype: int64
()
Atlanta stations missing data
Station USC00092283
False    12593
dtype: int64
()
Station USC00092485
False    12593
dtype: int64
()
Station USC00093060
False    12593
dtype: int64
()
Station USC00093621
False    12593
dtype: int64
()
Station USW00093842
False    12593
dtype: int64
()
Station USC00018469
False    12593
dtype: int64
()
Station USW00013871
False    12593
dtype: int64
()
San Francisco stations missing data
Station USC00043578
False    12773
dtype: int64
()
Station USC00045795
False    12773
dtype: int64
()
Station USC00047916
False    12773
dtype: int64
()
Station USC00049742
False    12773
dtype: int64
()
Station USW00023271
False    12773
dtype: int64
()
Station USW00023272
False    12

In [68]:
#Merge Datasets from each stock with PARAM_STOCK-NAME as default columns           
######Stock Data######
stocks = ['Apple, Inc Stock','International Business Machines Stock','Wal-Mart Stores, Inc Common St Stock','FedEx Corporation','The Boeing Company']
stock_data = pd.DataFrame()
for stock in stocks:
    path = 'Stock Data/%s.csv' % stock
    if os.path.exists(path):
        frame = pd.read_csv(path)
        frame.columns = ['DATE','OPEN_'+stock,'HIGH_'+stock,'LOW_'+stock,'CLOSE_'+stock,'VOLUME_'+stock,'ADJ CLOSE_'+stock]
        if stock_data.empty:
            stock_data = stock_data.append(frame,ignore_index=True)
        else:
            stock_data = stock_data.merge(frame, on='DATE', how='inner')
stock_data = stock_data.groupby('DATE').mean()
stock_data.reset_index(level=0, inplace=True)

In [98]:
pd.DataFrame.head(NYC_TMAX)

,TMAX_USC00300889,TMAX_USC00305426,TMAX_USW00094745,TMAX_USC00283951,TMAX_USW00014734
0,67,11,50,78,72
1,39,17,28,67,67
2,22,6,0,39,39
3,-17,-17,-44,33,-6
4,0,-33,-28,-22,6


In [ ]:
data_frames = [ATL_weather,CH_weather,NYC_weather,LA_weather,SD_weather,SF_weather,stock_data]
df_final = reduce(lambda left,right: pd.merge(left,right,how='inner',on='DATE'), data_frames)


    

In [100]:
NYC_TMAX = pd.DataFrame()
for column in list(NYC_weather.columns.values):
    if (re.match('TMAX',column)):
        NYC_TMAX[column] = NYC_weather[column]
print(NYC_TMAX.corr())

                  TMAX_USC00300889  TMAX_USC00305426  TMAX_USW00094745  \
TMAX_USC00300889          1.000000          0.949665          0.963172   
TMAX_USC00305426          0.949665          1.000000          0.969197   
TMAX_USW00094745          0.963172          0.969197          1.000000   
TMAX_USC00283951          0.925395          0.943673          0.917195   
TMAX_USW00014734          0.961987          0.967170          0.987811   

                  TMAX_USC00283951  TMAX_USW00014734  
TMAX_USC00300889          0.925395          0.961987  
TMAX_USC00305426          0.943673          0.967170  
TMAX_USW00094745          0.917195          0.987811  
TMAX_USC00283951          1.000000          0.916735  
TMAX_USW00014734          0.916735          1.000000  


In [101]:
NYC_TMIN = pd.DataFrame()
for column in list(NYC_weather.columns.values):
    if (re.match('TMIN',column)):
        NYC_TMIN[column] = NYC_weather[column]
print(NYC_TMIN.corr())

                  TMIN_USC00300889  TMIN_USC00305426  TMIN_USW00094745  \
TMIN_USC00300889          1.000000          0.938266          0.958629   
TMIN_USC00305426          0.938266          1.000000          0.969201   
TMIN_USW00094745          0.958629          0.969201          1.000000   
TMIN_USC00283951          0.954916          0.943035          0.956407   
TMIN_USW00014734          0.953476          0.973438          0.983732   

                  TMIN_USC00283951  TMIN_USW00014734  
TMIN_USC00300889          0.954916          0.953476  
TMIN_USC00305426          0.943035          0.973438  
TMIN_USW00094745          0.956407          0.983732  
TMIN_USC00283951          1.000000          0.959202  
TMIN_USW00014734          0.959202          1.000000  


In [102]:
NYC_SNOW = pd.DataFrame()
for column in list(NYC_weather.columns.values):
    if (re.match('SNOW',column)):
        NYC_SNOW[column] = NYC_weather[column]
print(NYC_SNOW.corr())

                  SNOW_USC00300889  SNOW_USC00305426  SNOW_USW00094745  \
SNOW_USC00300889          1.000000          0.419590          0.371333   
SNOW_USC00305426          0.419590          1.000000          0.453113   
SNOW_USW00094745          0.371333          0.453113          1.000000   
SNOW_USC00283951          0.552431          0.482675          0.435838   
SNOW_USW00014734          0.661293          0.473820          0.401526   

                  SNOW_USC00283951  SNOW_USW00014734  
SNOW_USC00300889          0.552431          0.661293  
SNOW_USC00305426          0.482675          0.473820  
SNOW_USW00094745          0.435838          0.401526  
SNOW_USC00283951          1.000000          0.471457  
SNOW_USW00014734          0.471457          1.000000  


In [103]:
NYC_SNWD = pd.DataFrame()
for column in list(NYC_weather.columns.values):
    if (re.match('SNWD',column)):
        NYC_SNWD[column] = NYC_weather[column]
print(NYC_SNWD.corr())

                  SNWD_USC00300889  SNWD_USC00305426  SNWD_USW00094745  \
SNWD_USC00300889          1.000000          0.441816          0.283565   
SNWD_USC00305426          0.441816          1.000000          0.311591   
SNWD_USW00094745          0.283565          0.311591          1.000000   
SNWD_USC00283951          0.707897          0.530704          0.393704   
SNWD_USW00014734          0.720874          0.507924          0.380402   

                  SNWD_USC00283951  SNWD_USW00014734  
SNWD_USC00300889          0.707897          0.720874  
SNWD_USC00305426          0.530704          0.507924  
SNWD_USW00094745          0.393704          0.380402  
SNWD_USC00283951          1.000000          0.905422  
SNWD_USW00014734          0.905422          1.000000  


In [104]:
NYC_PRCP = pd.DataFrame()
for column in list(NYC_weather.columns.values):
    if (re.match('PRCP',column)):
        NYC_PRCP[column] = NYC_weather[column]
print(NYC_PRCP.corr())

                  PRCP_USC00300889  PRCP_USC00305426  PRCP_USW00094745  \
PRCP_USC00300889          1.000000          0.508703          0.455534   
PRCP_USC00305426          0.508703          1.000000          0.575704   
PRCP_USW00094745          0.455534          0.575704          1.000000   
PRCP_USC00283951          0.472343          0.543111          0.411249   
PRCP_USW00014734          0.449696          0.523968          0.687953   

                  PRCP_USC00283951  PRCP_USW00014734  
PRCP_USC00300889          0.472343          0.449696  
PRCP_USC00305426          0.543111          0.523968  
PRCP_USW00094745          0.411249          0.687953  
PRCP_USC00283951          1.000000          0.326849  
PRCP_USW00014734          0.326849          1.000000  
